In [2]:
import getpass
import os
import torch
import numpy as np
from PIL import Image
import torchvision
from torch.autograd import Variable
from torch.utils.data import Dataset
from torchvision import transforms
import pandas as pd
import pydicom
import matplotlib.pyplot as plt
import torch.nn.functional as F
from collections import OrderedDict
#from skimage import transform
import sys
from sklearn.metrics import roc_curve, auc
import copy
from sklearn.metrics import f1_score
import torchvision.models as models
import re
import xml.etree.ElementTree as ET
import cv2
from matplotlib import rcParams, animation, rc
from ipywidgets import interact, interactive, fixed
from ipywidgets.widgets import *

#from display import read_image, draw_boxes, draw_grid, draw_text
#import display

In [3]:
# with this function you set the value of the environment variable CUDA_VISIBLE_DEVICES
# to set which GPU to use
# it also reserves this amount of memory for your exclusive use. This might be important for 
# not having other people using the resources you need in shared systems
# the homework was tested in a GPU with 4GB of memory, and running this function will require at least
# as much
# if you want to test in a GPU with less memory, you can call this function
# with the argument minimum_memory_mb specifying how much memory from the GPU you want to reserve
def define_gpu_to_use(minimum_memory_mb = 3800):
    gpu_to_use = None
    try: 
        os.environ['CUDA_VISIBLE_DEVICES']
        print('GPU already assigned before: ' + str(os.environ['CUDA_VISIBLE_DEVICES']))
        return
    except:
        pass
    torch.cuda.empty_cache()
    for i in range(16):
        free_memory = !nvidia-smi --query-gpu=memory.free -i $i --format=csv,nounits,noheader
        if free_memory[0] == 'No devices were found':
            break
        free_memory = int(free_memory[0])
        if free_memory>minimum_memory_mb-500:
            gpu_to_use = i
            break
    if gpu_to_use is None:
        print('Could not find any GPU available with the required free memory of ' +str(minimum_memory_mb) + 'MB. Please use a different system for this assignment.')
    else:
        os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_to_use)
        print('Chosen GPU: ' + str(gpu_to_use))
        x = torch.rand((256,1024,minimum_memory_mb-500)).cuda()
        x = torch.rand((1,1)).cuda()
        del x
define_gpu_to_use()

Chosen GPU: 0


In [4]:
def load_raw_images(root,split):
    set_of_transforms = {}
    set_of_transforms['train'] = transforms.Compose(
        [transforms.RandomRotation(30),
         transforms.RandomResizedCrop(224),
         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])
    set_of_transforms['test'] = transforms.Compose(
        [transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])
    set_of_transforms['validate'] = set_of_transforms['test']
    

    whole_dataset = torchvision.datasets.ImageFolder(root=root+'/'+split, transform=set_of_transforms[split])
    
    return whole_dataset   

def load_grayscale_images(root,split):
    set_of_transforms = {}
    set_of_transforms['train'] = transforms.Compose(
        [transforms.RandomRotation(30),
         transforms.RandomResizedCrop(224),
         transforms.RandomHorizontalFlip(),
         transforms.Grayscale(num_output_channels=3),
         transforms.ToTensor(), 
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])
    set_of_transforms['test'] = transforms.Compose(
        [transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])
    set_of_transforms['validate'] = set_of_transforms['test']
    

    whole_dataset = torchvision.datasets.ImageFolder(root=root+'/'+split, transform=set_of_transforms[split])
    
    return whole_dataset   

def find_classes(dir):
    classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx

#color images
path_to_img_folder = '../color/60_20_20'
train_data = load_raw_images(path_to_img_folder,'train')
test_data = load_raw_images(path_to_img_folder,'test')
val_data = load_raw_images(path_to_img_folder,'validate')

#grayscale images
train_grayscale_data = load_grayscale_images(path_to_img_folder,'train')
test_grayscale_data = load_grayscale_images(path_to_img_folder,'test')
val_grayscale_data = load_grayscale_images(path_to_img_folder,'validate')

#segmented images
path_to_segment_img_folder = '../segment/60_20_20segmented'
train_segmented_data = load_raw_images(path_to_segment_img_folder,'train')
test_segmented_data = load_raw_images(path_to_segment_img_folder,'test')
val_segmented_data = load_raw_images(path_to_segment_img_folder,'validate')


label_names,label_indices = find_classes('../color/60_20_20/train')
label_names_dict = dict((v,k) for k,v in label_indices.items())
print ("Length of the train set:{}".format(len(train_data)))
print ("Length of the test set:{}".format(len(test_data)))
print ("Length of the validation set:{}".format(len(val_data)))
print (" ")
print ("The label names along with its corresponding key:{}".format(label_names_dict))

Length of the train set:16051
Length of the test set:6453
Length of the validation set:4264
 
The label names along with its corresponding key:{0: 'Apple_Frogeye_Spot', 1: 'Apple___Apple_scab', 2: 'Apple___Cedar_apple_rust', 3: 'Apple___healthy', 4: 'Blueberry___healthy', 5: 'Cherry_including_sour___Powdery_mildew', 6: 'Cherry_including_sour___healthy', 7: 'Corn_maize___Cercospora_leaf_spot Gray_leaf_spot', 8: 'Corn_maize___Common_rust_', 9: 'Corn_maize___Northern_Leaf_Blight', 10: 'Corn_maize___healthy', 11: 'Grape___Black_rot', 12: 'Grape___Esca_Black_Measles', 13: 'Grape___Leaf_blight_Isariopsis_Leaf_Spot', 14: 'Grape___healthy', 15: 'Orange___Haunglongbing_Citrus_greening', 16: 'Peach___Bacterial_spot', 17: 'Peach___healthy', 18: 'Pepper_bell___Bacterial_spot', 19: 'Pepper_bell___healthy', 20: 'Potato___Early_blight', 21: 'Potato___Late_blight', 22: 'Potato___healthy', 23: 'Raspberry___healthy', 24: 'Soybean___healthy', 25: 'Squash___Powdery_mildew', 26: 'Strawberry___Leaf_scorch',

In [5]:
#original images (RGB) data loader
train_loader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size = 84, num_workers = 3)
test_loader = torch.utils.data.DataLoader(test_data, shuffle = True, batch_size = 84, num_workers = 3)
val_loader = torch.utils.data.DataLoader(val_data, shuffle = True, batch_size = 84, num_workers = 3)

#grayscale data loader
train_grayscale_loader = torch.utils.data.DataLoader(train_grayscale_data, shuffle = True, batch_size = 84, num_workers = 3)
test_grayscale_loader = torch.utils.data.DataLoader(test_grayscale_data, shuffle = True, batch_size = 84, num_workers = 3)
val_grayscale_loader = torch.utils.data.DataLoader(val_grayscale_data, shuffle = True, batch_size = 84, num_workers = 3)

#segmented images data loader
train_segmented_loader = torch.utils.data.DataLoader(train_segmented_data, shuffle = True, batch_size = 84, num_workers = 3)
test_segmented_loader = torch.utils.data.DataLoader(test_segmented_data, shuffle = True, batch_size = 84, num_workers = 3)
val_segmented_loader = torch.utils.data.DataLoader(val_segmented_data, shuffle = True, batch_size = 84, num_workers = 3)

#visualization
train_loader_vis = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size = 1, num_workers = 3)

print ("Number of batches train set:{}".format(len(train_loader)))
print ("Number of batches test set:{}".format(len(test_loader)))
print ("Number of batches validation set:{}".format(len(val_loader)))

Number of batches train set:192
Number of batches test set:77
Number of batches validation set:51


In [65]:
class ConvNet(torch.nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.convolution_layer_1 = torch.nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3)
        self.relu = torch.nn.ReLU()
        self.batchNormalization1 = torch.nn.BatchNorm2d(16)
        self.batchNormalization2 = torch.nn.BatchNorm2d(32)
        self.batchNormalization3 = torch.nn.BatchNorm2d(64)
        self.maxPooling_layer = torch.nn.MaxPool2d(kernel_size = 3)
        self.maxPooling_layer1 = torch.nn.MaxPool2d(kernel_size = 2)
        self.convolution_layer_2 = torch.nn.Conv2d(in_channels = 16 , out_channels = 32, kernel_size = 3)
        self.convolution_layer_3 = torch.nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3)
        self.convolution_layer_4 = torch.nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3)
        self.convolution_layer_5 = torch.nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3)
        self.dropout = torch.nn.Dropout(0.25)
        self.fully_connected_layer = torch.nn.Linear(in_features = 2304, out_features = 1024)
        self.fully_connected_layer1 = torch.nn.Linear(in_features = 1024, out_features = 38)

    def forward(self, x):
        x = self.convolution_layer_1(x)
        x = self.relu(x)
        #x = self.batchNormalization1(x)
        x = self.maxPooling_layer(x)
        #x = self.dropout(x)
        
        x = self.convolution_layer_2(x)
        x = self.relu(x)
        x = self.batchNormalization2(x)
        x = self.maxPooling_layer1(x)
        #x = self.dropout(x)
        
        x = self.convolution_layer_3(x)
        x = self.relu(x)
        #x = self.batchNormalization2(x)
        x = self.maxPooling_layer1(x)
        #x = self.dropout(x)
        
        x = self.convolution_layer_4(x)
        x = self.relu(x)
        x = self.batchNormalization3(x)
        
        x = self.convolution_layer_5(x)
        x = self.relu(x)
        #x = self.batchNormalization3(x)
        x = self.maxPooling_layer1(x)
        #x = self.dropout(x)
        
        x = x.view(x.size(0),-1)
        x = self.dropout(x)
        x = self.fully_connected_layer(x)
        x = self.fully_connected_layer1(x)
        return torch.nn.functional.log_softmax(x,dim=1)
      
classification_model = ConvNet()
#torch.cuda.empty_cache()
#model2 = model2.cuda()
print(classification_model)

ConvNet(
  (convolution_layer_1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (relu): ReLU()
  (batchNormalization1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchNormalization2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchNormalization3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxPooling_layer): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (maxPooling_layer1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (convolution_layer_2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (convolution_layer_3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (convolution_layer_4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (convolution_layer_5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout): Dropout(p=0.25)
  (fully_connected_layer): Linear(in_features=2304, out_featu

In [66]:
def train_classification_model(data_loader, model, criterion, optimizer, lr_decay, num_epochs,mode='train'):
    for epoch in range(num_epochs):
        model.train()
        loss_sum = []; acc = 0
        for i in data_loader:
            input_img =i[0].cuda()
            target = i[1].cuda() 
            if mode=='train':
                optimizer.zero_grad()
            predicted_output = model(input_img)
            loss = criterion(predicted_output, target)
            _,pred_out = torch.max(predicted_output,1)    
            if mode=='train':
                loss.backward()
                optimizer.step()
            loss_sum.append(loss.item())
            acc+=torch.sum(pred_out==target.data)
        epoch_loss = sum(loss_sum)/len(loss_sum)
        epoch_accuracy = acc.item()/len(train_data)
        print ("epoch:"+str(epoch))
        print ("Loss:{:.6f}".format(epoch_loss))
        print ("Accuracy:{:.6f}".format(epoch_accuracy))
        print (" ")



In [67]:
#test models
def test_model(model,data_loader):
    model.eval()
    accuracy_list=[]
    with torch.no_grad():
        predicted_output_concat = np.zeros([84])
        target_concat = np.zeros([84])
        for i in data_loader:
            input_img=i[0].cuda()
            #print (input_img[0].shape)
            target=i[1]
            target=target.cuda()
            pred_output = model(input_img)
            eq = (target.data == torch.exp(pred_output).max(1)[1])
            pred_out = torch.exp(pred_output).max(1)[1]
            accuracy_list.append(eq.type_as(torch.FloatTensor()).mean())
            predicted_output_concat = np.concatenate((predicted_output_concat, pred_out.cpu().detach().numpy()), axis = 0)
            target_concat = np.concatenate((target_concat, target.cpu().detach().numpy()), axis = 0)
        print ("Test accuracy:{:.6f}".format(sum(accuracy_list)/len(accuracy_list)))
        #print (target_concat)
        #print (target_concat.shape)
        #print (predicted_output_concat)
        #print (predicted_output_concat.shape)
    print ("Macro F1 score:{}".format(f1_score(target_concat, predicted_output_concat, average="macro")))
    print ("Micro F1 score:{}".format(f1_score(target_concat, predicted_output_concat, average="micro")))


## Adam and 0.001

### Model accuracies on color datatset

In [19]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.Adam(classification_model.parameters(), lr=0.001)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:0.700417
Accuracy:0.792785
 
epoch:1
Loss:0.679126
Accuracy:0.795215
 
epoch:2
Loss:0.714455
Accuracy:0.791477
 
epoch:3
Loss:0.630987
Accuracy:0.807613
 
epoch:4
Loss:0.597447
Accuracy:0.818454
 
epoch:5
Loss:0.572220
Accuracy:0.828796
 
epoch:6
Loss:0.577957
Accuracy:0.827923
 
epoch:7
Loss:0.574554
Accuracy:0.826304
 
epoch:8
Loss:0.532445
Accuracy:0.836895
 
epoch:9
Loss:0.533674
Accuracy:0.840758
 


#### Validation Accuracy

In [20]:
test_model(classification_model, val_loader)

Test accuracy:0.857872
Macro F1 score:0.8181735264432933
Micro F1 score:0.860395584176633


#### Test Accuracy

In [21]:
test_model(classification_model, test_loader)

Test accuracy:0.863233
Macro F1 score:0.82222569502911
Micro F1 score:0.8650757228086278


### Model accuracies on segmented dataset

In [37]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.Adam(classification_model.parameters(), lr=0.001)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_segmented_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:2.205165
Accuracy:0.400037
 
epoch:1
Loss:1.593978
Accuracy:0.542458
 
epoch:2
Loss:1.369356
Accuracy:0.598903
 
epoch:3
Loss:1.266563
Accuracy:0.628434
 
epoch:4
Loss:1.175051
Accuracy:0.651112
 
epoch:5
Loss:1.071412
Accuracy:0.680082
 
epoch:6
Loss:1.043254
Accuracy:0.689241
 
epoch:7
Loss:0.976179
Accuracy:0.706249
 
epoch:8
Loss:0.970745
Accuracy:0.708865
 
epoch:9
Loss:0.919327
Accuracy:0.724628
 


#### Validation accuracy

In [38]:
test_model(classification_model, val_segmented_loader)

Test accuracy:0.786287
Macro F1 score:0.7158673769954519
Micro F1 score:0.7901518637827888


#### Test accuracy

In [39]:
test_model(classification_model, test_segmented_loader)

Test accuracy:0.777770
Macro F1 score:0.7085493876514524
Micro F1 score:0.7808575803981623


### Model accuracies on grayscale dataset

In [68]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.Adam(classification_model.parameters(), lr=0.001)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_grayscale_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:2.502543
Accuracy:0.320915
 
epoch:1
Loss:1.956925
Accuracy:0.448196
 
epoch:2
Loss:1.768757
Accuracy:0.498723
 
epoch:3
Loss:1.599823
Accuracy:0.538720
 
epoch:4
Loss:1.479176
Accuracy:0.573921
 
epoch:5
Loss:1.393083
Accuracy:0.599215
 
epoch:6
Loss:1.350538
Accuracy:0.607003
 
epoch:7
Loss:1.301595
Accuracy:0.616223
 
epoch:8
Loss:1.236184
Accuracy:0.634478
 
epoch:9
Loss:1.202500
Accuracy:0.646439
 


#### Validation Accuracy

In [69]:
test_model(classification_model, val_grayscale_loader)

Test accuracy:0.629844
Macro F1 score:0.5509806852717541
Micro F1 score:0.6373045078196872


#### Test Accuracy

In [70]:
test_model(classification_model, test_grayscale_loader)

Test accuracy:0.632814
Macro F1 score:0.5666033094094647
Micro F1 score:0.6377543215542297


## Adam and 0.0001

### Model accuracies on colored dataset

In [9]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.Adam(classification_model.parameters(), lr=0.0001)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:2.310437
Accuracy:0.380412
 
epoch:1
Loss:1.681469
Accuracy:0.526634
 
epoch:2
Loss:1.426972
Accuracy:0.585073
 
epoch:3
Loss:1.257546
Accuracy:0.630677
 
epoch:4
Loss:1.137535
Accuracy:0.664445
 
epoch:5
Loss:1.063591
Accuracy:0.685627
 
epoch:6
Loss:0.999239
Accuracy:0.702635
 
epoch:7
Loss:0.951057
Accuracy:0.712105
 
epoch:8
Loss:0.905986
Accuracy:0.725749
 
epoch:9
Loss:0.860122
Accuracy:0.745686
 


#### Validation Accuracy

In [11]:
test_model(classification_model, val_loader)

Test accuracy:0.817592
Macro F1 score:0.7565591310175958
Micro F1 score:0.8210671573137075


#### Test Accuracy

In [13]:
test_model(classification_model, test_loader)

Test accuracy:0.820783
Macro F1 score:0.7590345739015966
Micro F1 score:0.8230074957931773


### Model Accuracies on segmented dataset

In [18]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.Adam(classification_model.parameters(), lr=0.0001)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_segmented_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:2.353862
Accuracy:0.371130
 
epoch:1
Loss:1.731928
Accuracy:0.503458
 
epoch:2
Loss:1.516334
Accuracy:0.562083
 
epoch:3
Loss:1.394320
Accuracy:0.588437
 
epoch:4
Loss:1.296873
Accuracy:0.620397
 
epoch:5
Loss:1.230812
Accuracy:0.634851
 
epoch:6
Loss:1.160845
Accuracy:0.653604
 
epoch:7
Loss:1.098012
Accuracy:0.671298
 
epoch:8
Loss:1.064501
Accuracy:0.680020
 
epoch:9
Loss:1.010322
Accuracy:0.692480
 


#### Validation Accuracy

In [19]:
test_model(classification_model, val_segmented_loader)

Test accuracy:0.742304
Macro F1 score:0.6573911921578117
Micro F1 score:0.7471237919926369


#### Test Accuracy

In [20]:
test_model(classification_model, test_segmented_loader)

Test accuracy:0.735826
Macro F1 score:0.6664958612540932
Micro F1 score:0.7392036753445637


### Model Accuracies on grayscale dataset

In [25]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.Adam(classification_model.parameters(), lr=0.0001)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_grayscale_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:2.667838
Accuracy:0.290075
 
epoch:1
Loss:2.153155
Accuracy:0.406081
 
epoch:2
Loss:1.945169
Accuracy:0.459473
 
epoch:3
Loss:1.813996
Accuracy:0.488755
 
epoch:4
Loss:1.712136
Accuracy:0.514174
 
epoch:5
Loss:1.633236
Accuracy:0.537474
 
epoch:6
Loss:1.588259
Accuracy:0.548626
 
epoch:7
Loss:1.529038
Accuracy:0.561336
 
epoch:8
Loss:1.472828
Accuracy:0.575229
 
epoch:9
Loss:1.434384
Accuracy:0.587066
 


#### Validation Accuracy

In [26]:
test_model(classification_model, val_grayscale_loader)

Test accuracy:0.646840
Macro F1 score:0.5261404522202555
Micro F1 score:0.6534038638454461


/home/u1136693/env_dir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#### Test Accuracy

In [27]:
test_model(classification_model, test_grayscale_loader)

Test accuracy:0.632895
Macro F1 score:0.514897820559482
Micro F1 score:0.6376013461832645


## SGD + Momentum and 0.001

### Model Accuracies on colored dataset

In [31]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.SGD(classification_model.parameters(), lr=0.001, momentum=0.9)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:2.898652
Accuracy:0.254688
 
epoch:1
Loss:2.245950
Accuracy:0.404648
 
epoch:2
Loss:1.915377
Accuracy:0.474051
 
epoch:3
Loss:1.649906
Accuracy:0.535107
 
epoch:4
Loss:1.481189
Accuracy:0.579279
 
epoch:5
Loss:1.338982
Accuracy:0.613918
 
epoch:6
Loss:1.202439
Accuracy:0.645692
 
epoch:7
Loss:1.112736
Accuracy:0.673665
 
epoch:8
Loss:1.040124
Accuracy:0.690175
 
epoch:9
Loss:0.972480
Accuracy:0.709364
 


#### Validation Accuracy

In [33]:
test_model(classification_model, val_loader)

Test accuracy:0.697187
Macro F1 score:0.6122174186503123
Micro F1 score:0.703081876724931


#### Test Accuracy

In [34]:
test_model(classification_model, test_loader)

Test accuracy:0.685488
Macro F1 score:0.5907739018389954
Micro F1 score:0.6893070215695273


### Model Accuracies on segemented dataset

In [38]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.SGD(classification_model.parameters(), lr=0.001, momentum=0.9)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_segmented_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:2.921082
Accuracy:0.249517
 
epoch:1
Loss:2.331580
Accuracy:0.374058
 
epoch:2
Loss:2.059590
Accuracy:0.430440
 
epoch:3
Loss:1.822367
Accuracy:0.482338
 
epoch:4
Loss:1.637186
Accuracy:0.529126
 
epoch:5
Loss:1.510697
Accuracy:0.552427
 
epoch:6
Loss:1.419749
Accuracy:0.581210
 
epoch:7
Loss:1.346216
Accuracy:0.601209
 
epoch:8
Loss:1.260093
Accuracy:0.625444
 
epoch:9
Loss:1.205662
Accuracy:0.637655
 


#### Validation Accuracy

In [39]:
test_model(classification_model, val_segmented_loader)

Test accuracy:0.686455
Macro F1 score:0.5667361039253492
Micro F1 score:0.6930510814542108


#### Test Accuracy

In [40]:
test_model(classification_model, test_segmented_loader)

Test accuracy:0.681130
Macro F1 score:0.5619488126329447
Micro F1 score:0.6852986217457887


### Model Accuracies on grayscale dataset

In [44]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.SGD(classification_model.parameters(), lr=0.001, momentum=0.9)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_grayscale_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:3.083138
Accuracy:0.196125
 
epoch:1
Loss:2.602838
Accuracy:0.300542
 
epoch:2
Loss:2.401055
Accuracy:0.349262
 
epoch:3
Loss:2.216840
Accuracy:0.386082
 
epoch:4
Loss:2.094850
Accuracy:0.420223
 
epoch:5
Loss:2.016675
Accuracy:0.442278
 
epoch:6
Loss:1.931963
Accuracy:0.460096
 
epoch:7
Loss:1.875110
Accuracy:0.470812
 
epoch:8
Loss:1.780551
Accuracy:0.496106
 
epoch:9
Loss:1.738123
Accuracy:0.510311
 


#### Validation Accuracy

In [45]:
test_model(classification_model, val_grayscale_loader)

Test accuracy:0.594961
Macro F1 score:0.4294813730969562
Micro F1 score:0.6025758969641214


#### Test Accuracy

In [46]:
test_model(classification_model, test_grayscale_loader)

Test accuracy:0.586466
Macro F1 score:0.41904708351464925
Micro F1 score:0.5918617102646474


## SGD + Momentum and 0.0001

### Model Accuracies on colored dataset

In [50]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.SGD(classification_model.parameters(), lr=0.0001, momentum=0.9)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:3.536460
Accuracy:0.110897
 
epoch:1
Loss:3.287258
Accuracy:0.181796
 
epoch:2
Loss:3.030194
Accuracy:0.230204
 
epoch:3
Loss:2.882275
Accuracy:0.255000
 
epoch:4
Loss:2.777023
Accuracy:0.271634
 
epoch:5
Loss:2.673358
Accuracy:0.305651
 
epoch:6
Loss:2.571978
Accuracy:0.331630
 
epoch:7
Loss:2.479977
Accuracy:0.352751
 
epoch:8
Loss:2.390455
Accuracy:0.372500
 
epoch:9
Loss:2.318445
Accuracy:0.383901
 


#### Validation Accuracy

In [51]:
test_model(classification_model, val_loader)

Test accuracy:0.375875
Macro F1 score:0.17184798723391512
Micro F1 score:0.3882244710211591


#### Test Accuracy

In [52]:
test_model(classification_model, test_loader)

Test accuracy:0.387775
Macro F1 score:0.175344178285155
Micro F1 score:0.3959002600581306


### Model Accuracies on segmented dataset

In [56]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.SGD(classification_model.parameters(), lr=0.0001, momentum=0.9)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_segmented_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:3.496635
Accuracy:0.117687
 
epoch:1
Loss:3.225508
Accuracy:0.205096
 
epoch:2
Loss:3.015645
Accuracy:0.233256
 
epoch:3
Loss:2.875058
Accuracy:0.248770
 
epoch:4
Loss:2.779653
Accuracy:0.272506
 
epoch:5
Loss:2.689734
Accuracy:0.294312
 
epoch:6
Loss:2.605827
Accuracy:0.314809
 
epoch:7
Loss:2.545349
Accuracy:0.323718
 
epoch:8
Loss:2.494230
Accuracy:0.334808
 
epoch:9
Loss:2.444059
Accuracy:0.347455
 


#### Validation Accuracy

In [57]:
test_model(classification_model, val_segmented_loader)

Test accuracy:0.359485
Macro F1 score:0.1454121579791586
Micro F1 score:0.3722963644730787


#### Test Accuracy

In [58]:
test_model(classification_model, test_segmented_loader)

Test accuracy:0.365951
Macro F1 score:0.14705508710769
Micro F1 score:0.37457886676875957


### Model Accuracies on grayscale dataset

In [62]:
#train
criterion = torch.nn.NLLLoss()
classification_model = classification_model.cuda()
optimizer = torch.optim.SGD(classification_model.parameters(), lr=0.0001, momentum=0.9)
#lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
lr_schedule = 0 
train_classification_model(train_grayscale_loader, classification_model, criterion, optimizer,lr_schedule,10,'train')

epoch:0
Loss:3.527983
Accuracy:0.112205
 
epoch:1
Loss:3.280571
Accuracy:0.173696
 
epoch:2
Loss:3.125444
Accuracy:0.185160
 
epoch:3
Loss:3.048638
Accuracy:0.199738
 
epoch:4
Loss:2.987383
Accuracy:0.213632
 
epoch:5
Loss:2.916442
Accuracy:0.231325
 
epoch:6
Loss:2.855892
Accuracy:0.250265
 
epoch:7
Loss:2.793217
Accuracy:0.262414
 
epoch:8
Loss:2.723363
Accuracy:0.278861
 
epoch:9
Loss:2.674181
Accuracy:0.287396
 


#### Validation Accuracy

In [63]:
test_model(classification_model, val_grayscale_loader)

Test accuracy:0.283818
Macro F1 score:0.10941926255540392
Micro F1 score:0.29737810487580496


#### Test Accuracy

In [64]:
test_model(classification_model, test_grayscale_loader)

Test accuracy:0.294218
Macro F1 score:0.1117551411779298
Micro F1 score:0.30319718525317424
